<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/getting-started/01-hello-world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AADC basic "Hello World" example
This example demonstrates how to use the AADC library to record a simple function and calculate its derivatives

In [1]:
import sys
!pip install https://matlogica.com/DemoReleases/aadc-1.7.5.2401-cp3{sys.version_info.minor}-cp3{sys.version_info.minor}-linux_x86_64.whl

  Using cached https://matlogica.com/DemoReleases/aadc-1.7.5.2401-cp311-cp311-linux_x86_64.whl (35.1 MB)


In [2]:
import aadc
import numpy as np

# Simple function to record
The function can take arbitrary input types and can be defined in external file

In [3]:
def F(x,y,z):
    return np.exp(x / y + z)

# Object to hold recorded AADC Kernels
allow to speed up the calculation of the function itself and AAD for its derivatives


In [4]:
funcs = aadc.Functions()

# Define function arguments.
Note you need to use aadc.idouble instead of float

In [5]:
x = aadc.idouble(1.0)
y = aadc.idouble(2.0)
z = aadc.idouble(3.0)

# Trigger recording of the function

In [6]:
funcs.start_recording()

You are using evaluation version of AADC. Expire date is 20240901


# Mark the function arguments as input and save reference argument ids

In [7]:
xArg = x.mark_as_input()
yArg = y.mark_as_input()
zArg = z.mark_as_input()

# Call the function and add some operations

In [8]:
f = F(x,y,z) + x

# Mark the result as output

In [9]:
fRes = f.mark_as_output()

# Stop recording
After this step AADC Kernel containing native machine CPU code will be generated and stored in the funcs object

In [10]:
funcs.stop_recording()

# Check if the function is recorded properly
So it can be used for arbitrary input values. This should return 0 if everything is OK, indicating that no branches in the function are not supported.


In [11]:
funcs.print_passive_extract_locations()

Number active to passive conversions: 0 while recording Python


# New samples of input values
To calculate the function and its derivatives at. Note that the x input is a vector of 20 samples and the y and z are scalars

In [12]:
inputs = {
    xArg:(1.0 * np.random.normal(1, 0.2, 20)),
    yArg:(2.0),
    zArg:(3.0),
}

Key: what output, value: what gradients are needed

In [13]:
request = {fRes:[xArg,yArg,zArg]}  

Run AADC Kernel for array inputs using 4 CPU threads and avx2

In [14]:
Res = aadc.evaluate(funcs, request, inputs, aadc.ThreadPool(4))

## Function output

In [15]:
Res[0][fRes]

array([31.44085708, 36.1707629 , 35.76464723, 33.14819859, 34.54444068,
       30.77353102, 36.60770188, 29.06156381, 31.56890252, 33.02241089,
       36.09123998, 32.05047647, 33.73621372, 38.20971796, 36.93638856,
       38.89931825, 38.33845253, 33.87274325, 45.91800075, 35.17406739])

## df/dx

In [16]:
Res[1][fRes][xArg]

array([16.29946359, 18.52841121, 18.33636984, 17.10200812, 17.76007367,
       15.9864705 , 18.73515493, 15.18542004, 16.35956563, 17.04279822,
       18.4907979 , 16.58573498, 17.37896198, 19.49425183, 18.89076258,
       19.8215115 , 19.55532241, 17.44330598, 23.16723777, 18.05730779])

## df/dy

In [17]:
Res[1][fRes][yArg]


array([ -6.44053791,  -9.76280344,  -9.46485661,  -7.6016159 ,
        -8.58361594,  -5.99900933, -10.08591176,  -4.89910306,
        -6.5260587 ,  -7.51456253,  -9.70428103,  -6.84998121,
        -8.01168548, -11.29272244, -10.33069351, -11.82268774,
       -11.39118388,  -8.10762925, -17.55118989,  -9.03569776])